In [ ]:
import requests
import base64
import json
from openai import OpenAI

# Document Parser API로 PDF 텍스트 추출
def parse_document(file_path):
    api_key = "YOUR_DOCUMENT_API_KEY"
    url = "https://api.upstage.ai/v1/document-ai/document-parse"
    headers = {"Authorization": f"Bearer {api_key}"}
    files = {"document": open(file_path, "rb")}
    data = {
        "ocr": "force",
        "coordinates": True,
        "chart_recognition": True,
        "output_formats": "['text']",
        "base64_encoding": "['table']",
        "model": "document-parse"
    }
    response = requests.post(url, headers=headers, files=files, data=data)
    if response.status_code == 200:
        return response.json()['text']
    return None

# Information Extract API로 PNG에서 정보 추출
def extract_info_from_image(img_path):
    client = OpenAI(
        api_key="up_IANAGmcf5Z0MK40wgyiNBYJAlFMTe",
        base_url="https://api.upstage.ai/v1/information-extraction"
    )
    with open(img_path, 'rb') as img_file:
        base64_data = base64.b64encode(img_file.read()).decode('utf-8')

    response = client.chat.completions.create(
        model="information-extract",
        messages=[{
            "role": "user",
            "content": [{
                "type": "image_url",
                "image_url": {"url": f"data:image/png;base64,{base64_data}"}
            }]
        }],
        response_format={
            "type": "json_schema",
            "json_schema": {
                "name": "bitcoin_data",
                "schema": {
                    "type": "object",
                    "properties": {
                        "price": {"type": "string", "description": "비트코인 가격"},
                        "date": {"type": "string", "description": "날짜"}
                    },
                    "required": ["price"]
                }
            }
        }
    )
    return json.loads(response.choices[0].message.content)

# 테스트
if __name__ == "__main__":
    pdf_text = parse_document("bitcoin_report.pdf")
    img_data = extract_info_from_image("bitcoin_chart.png")
    print("PDF Text:", pdf_text)
    print("Image Data:", img_data)

In [ ]:
import re
from textblob import TextBlob

# PDF 텍스트에서 가격 추출
def extract_price_data(text):
    prices = re.findall(r'\$\d+(?:,\d+)?', text)
    return [float(price.replace('$', '').replace(',', '')) for price in prices]

# PNG 이미지 데이터 분석
def analyze_image_data(img_data):
    price = img_data.get("price", "0").replace('$', '').replace(',', '')
    return float(price) if price else 0

# 감정 분석 (PDF 텍스트 기반)
def analyze_sentiment(text):
    analysis = TextBlob(text)
    return analysis.sentiment.polarity  # -1(부정) ~ 1(긍정)

# 매매 신호 생성
def generate_signal(prices, img_price, sentiment_score):
    avg_price = sum(prices) / len(prices) if prices else img_price
    if sentiment_score > 0.5 and img_price > avg_price:  # 긍정적 + 가격 상승
        return "매수"
    elif sentiment_score < 0 and img_price < avg_price:  # 부정적 + 가격 하락
        return "매도"
    return "관망"

# 테스트
if __name__ == "__main__":
    pdf_text = "Bitcoin price is $60,000 and rising."
    img_data = {"price": "$62,000", "date": "2025-03-30"}
    prices = extract_price_data(pdf_text)
    img_price = analyze_image_data(img_data)
    sentiment = analyze_sentiment(pdf_text)
    signal = generate_signal(prices, img_price, sentiment)
    print("Signal:", signal)

In [ ]:
class TradingDecision:
    def __init__(self, balance=1000, btc_held=0):
        self.balance = balance
        self.btc_held = btc_held

    def make_decision(self, signal, current_price):
        if signal == "매수" and self.balance >= current_price * 0.1:
            amount = 0.1
            cost = amount * current_price
            self.balance -= cost
            self.btc_held += amount
            return f"0.1 BTC 매수 at ${current_price}"
        elif signal == "매도" and self.btc_held >= 0.1:
            amount = 0.1
            revenue = amount * current_price
            self.balance += revenue
            self.btc_held -= amount
            return f"0.1 BTC 매도 at ${current_price}"
        return "관망"

# 테스트
if __name__ == "__main__":
    trader = TradingDecision(balance=1000, btc_held=0.5)
    decision = trader.make_decision("매수", 62000)
    print("Decision:", decision)

In [ ]:
class TradingExecutor:
    def __init__(self):
        self.simulated = True

    def execute_trade(self, decision):
        if self.simulated:
            return f"시뮬레이션: {decision} 실행 완료"
        return "실제 거래 미구현"

# 테스트
if __name__ == "__main__":
    executor = TradingExecutor()
    result = executor.execute_trade("0.1 BTC 매수 at $62000")
    print("Result:", result)

In [ ]:
class FeedbackLoop:
    def __init__(self):
        self.trade_history = []

    def log_trade(self, decision, current_price, new_price):
        profit = 0
        if "매수" in decision:
            profit = (new_price - current_price) * 0.1
        elif "매도" in decision:
            profit = (current_price - new_price) * 0.1
        self.trade_history.append({"decision": decision, "profit": profit})

    def evaluate(self):
        total_profit = sum(trade["profit"] for trade in self.trade_history)
        return f"총 수익: ${total_profit:.2f}"

# 테스트
if __name__ == "__main__":
    feedback = FeedbackLoop()
    feedback.log_trade("0.1 BTC 매수 at $62000", 62000, 63000)
    print(feedback.evaluate())

In [ ]:
# 데이터 수집
pdf_text = parse_document("bitcoin_report.pdf")
img_data = extract_info_from_image("bitcoin_chart.png")

# 데이터 분석
prices = extract_price_data(pdf_text)
img_price = analyze_image_data(img_data)
sentiment = analyze_sentiment(pdf_text)
signal = generate_signal(prices, img_price, sentiment)

# 의사결정
trader = TradingDecision(balance=1000, btc_held=0)
decision = trader.make_decision(signal, img_price)

# 실행
executor = TradingExecutor()
result = executor.execute_trade(decision)

# 피드백
feedback = FeedbackLoop()
feedback.log_trade(decision, img_price, img_price + 1000)  # 가격 변화 가정
print("Execution Result:", result)
print(feedback.evaluate())